In [117]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [118]:
!pip install missingpy

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
import math

In [120]:
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [121]:
import sys
import numpy as np
import pandas as pd

from missingpy import MissForest
from dsan import Imputer as DSANImputer
from utils import *

import warnings
warnings.filterwarnings('ignore')

In [122]:
seed = 128
set_seed(seed)

In [123]:
def get_imputer(config):
    
    model = config['model']
    
    if model == 'DSAN':
        imputer = DSANImputer(rep_dim=config['rep_dim'],
                num_heads=config['num_heads'],
                n_hidden=config['n_hidden'],
                lr=config['lr'],
                weight_decay=config['weight_decay'],
                batch_size=config['batch_size'],
                epochs=config['epochs'],
                noise_percent=config['noise_percent'],
                stopped_epoch=config['stopped_epoch'])
    
    elif model == 'MissForest':
        imputer = MissForest(random_state=0)
        
    elif model == 'Statistics':
        imputer = None
        
    return imputer

In [124]:
'''def experiment_run(data_name, config, test=False):
    
    # Retrieve dataset
    dataset = get_data(data_name)
    print(f"Retrieved dataset for {data_name}")

    # Limit to 1000 samples if test flag is true
    if test:
        dataset = (dataset[0][:1000], dataset[1], dataset[2], dataset[3])
        print("Limited dataset to 1000 samples for testing")

    k = 5
    percent_missing_lst = [p for p in range(5, 25, 5)]    
    X_origin, n_col, num_vars, cat_vars = dataset

    # Apply category mapping
    X_origin = category_mapping(X_origin, cat_vars)
    print("Applied category mapping")

    # Print data size
    print("Data Size: {}".format(X_origin.shape))

    n_sample = X_origin.shape[0]
    idx_lst = list(range(n_sample))
    np.random.shuffle(idx_lst)

    # k-fold validation setup
    n_valid = len(idx_lst) // k
    print(f"Starting k-fold validation with k={k}")

    results = {}
    for percent_missing in percent_missing_lst:
        result = []
        print(f"\nProcessing {percent_missing}% missing data")

        for it, n in enumerate(range(0, n_sample, n_valid)):
            print(f"\nFold {it + 1}")
            valid_idx = idx_lst[n: n+n_valid]

            if len(valid_idx) < n_valid:
                print("Insufficient data for validation, breaking loop")
                break

            train_idx = list(set(idx_lst) - set(valid_idx))
            X = X_origin[train_idx, :]
            X_test = X_origin[valid_idx, :]

            # Generating incomplete data
            missing_mask = generate_missing_mask(X, percent_missing=percent_missing)
            X_incomplete = X.copy()
            X_incomplete[missing_mask] = np.nan
            print("Generated incomplete data with missing values")

            # Imputation
            imputer = get_imputer(config)
            X_imputed = imputer.fit_transform(X_incomplete, cat_vars=cat_vars)
            print("Completed imputation")
            
            # Print a small subset of original and imputed data for comparison
            print("Original data with missing values (subset):")
            print(X_incomplete[:5])  # Adjust the number of rows as needed
            #print("Imputed data (subset):")
            #print(X_imputed[:5])  # Adjust the number of rows as needed


            X_res = X.copy()
            X_res[missing_mask] = X_imputed[missing_mask]
            
            print("X_res with imputed values (subset):")
            print(X_res[:5]) 

            # Calculate metrics
            metric = cal_metric_numpy(X_res, X, missing_mask, num_vars, cat_vars)
            score = ex_classify(data_name=data_name, train_array=X_res, test_array=X_test, num_vars=num_vars, cat_vars=cat_vars)
            
            metric['clf_aucroc'] = round(score, 4)
            result.append(metric)
            print("Metrics for this fold:", metric)

        # Average results
        avg_result = dict()
        for key in result[0].keys():
            value = np.mean([m[key] for m in result])
            avg_result[key] = value 
            print(f"Average {key} for {percent_missing}% missing data: {value}")

        results[percent_missing] = avg_result

    print("Completed all iterations")
    return results
'''

'def experiment_run(data_name, config, test=False):\n    \n    # Retrieve dataset\n    dataset = get_data(data_name)\n    print(f"Retrieved dataset for {data_name}")\n\n    # Limit to 1000 samples if test flag is true\n    if test:\n        dataset = (dataset[0][:1000], dataset[1], dataset[2], dataset[3])\n        print("Limited dataset to 1000 samples for testing")\n\n    k = 5\n    percent_missing_lst = [p for p in range(5, 25, 5)]    \n    X_origin, n_col, num_vars, cat_vars = dataset\n\n    # Apply category mapping\n    X_origin = category_mapping(X_origin, cat_vars)\n    print("Applied category mapping")\n\n    # Print data size\n    print("Data Size: {}".format(X_origin.shape))\n\n    n_sample = X_origin.shape[0]\n    idx_lst = list(range(n_sample))\n    np.random.shuffle(idx_lst)\n\n    # k-fold validation setup\n    n_valid = len(idx_lst) // k\n    print(f"Starting k-fold validation with k={k}")\n\n    results = {}\n    for percent_missing in percent_missing_lst:\n        resul

In [126]:
def experiment_run(data_name, config, test=False):
    # Retrieve dataset
    dataset = get_data(data_name)
    print(f"Retrieved dataset for {data_name}")

    # Limit to 1000 samples if test flag is true
    if test:
        dataset = (dataset[0][:1000], dataset[1], dataset[2], dataset[3])
        print("Limited dataset to 1000 samples for testing")

    k = 5
    percent_missing_lst = [p for p in range(5, 25, 5)]
    X_origin, n_col, num_vars, cat_vars = dataset

   # Apply category mapping
    X_origin = category_mapping(X_origin, cat_vars)
    print("Applied category mapping")

    # Check if X_origin is modified in-place or replaced
    if X_origin is not None:
        print("X_origin is replaced with a new version where categorical variables have been mapped to numerical values.")
    else:
        print("X_origin is modified in-place, with categorical variables mapped to numerical values.")

    # Print data size
    print("Data Size: {}".format(X_origin.shape))


    n_sample = X_origin.shape[0]
    idx_lst = list(range(n_sample))
    np.random.shuffle(idx_lst)

    # k-fold validation setup
    n_valid = len(idx_lst) // k
    print(f"Starting k-fold validation with k={k}")

    results = {}
    for percent_missing in percent_missing_lst:
        result = []
        print(f"\nProcessing {percent_missing}% missing data")

        for it, n in enumerate(range(0, n_sample, n_valid)):
            print(f"\nFold {it + 1}")
            valid_idx = idx_lst[n: n+n_valid]

            if len(valid_idx) < n_valid:
                print("Insufficient data for validation, breaking loop")
                break

            train_idx = list(set(idx_lst) - set(valid_idx))
            X = X_origin[train_idx, :]
            X_test = X_origin[valid_idx, :]

            # Generating incomplete data
            missing_mask = generate_missing_mask(X, percent_missing=percent_missing)
            X_incomplete = X.copy()
            X_incomplete[missing_mask] = np.nan
            print("Generated incomplete data with missing values for fold {}:".format(it+1))
            print("Original data with missing values (subset):")
            print(X_incomplete[:5])  # Adjust the number of rows as needed

            # Imputation
            imputer = get_imputer(config)
            X_imputed = imputer.fit_transform(X_incomplete, cat_vars=cat_vars)
            print("Completed imputation for fold {}:".format(it+1))

            X_res = X.copy()
            X_res[missing_mask] = X_imputed[missing_mask]
            print("X_res with imputed values (subset):")
            print(X_res[:5])  # Adjust the number of rows as needed

            # Calculate metrics
            metric = cal_metric_numpy(X_res, X, missing_mask, num_vars, cat_vars)
            score = ex_classify(data_name=data_name, train_array=X_res, test_array=X_test, num_vars=num_vars, cat_vars=cat_vars)
            
            metric['clf_aucroc'] = round(score, 4)
            result.append(metric)
            print("Metrics for this fold:", metric)

        # Average results
        avg_result = dict()
        for key in result[0].keys():
            value = np.mean([m[key] for m in result])
            avg_result[key] = value 
            print(f"Average {key} for {percent_missing}% missing data: {value}")

        results[percent_missing] = avg_result

    print("Completed all iterations")
    return results


In [127]:
config = {
    'model': 'DSAN',
    'rep_dim': 32,
    'num_heads': 8,
    'n_hidden': 2,
    'lr': 3e-3,
    'weight_decay': 1e-5,
    'batch_size': 128,
    'epochs': 15,
    'noise_percent': 10,
    'stopped_epoch': 10
}

In [128]:
import missingpy
from missingpy import MissForest

In [129]:
data_name_lst = ['adult']

result_dir = './results1'
os.makedirs(result_dir)

for data_name in data_name_lst:
    results = experiment_run(data_name, config, test=True)
    results['config'] = config
    results['dataset'] = data_name
    result_path = os.path.join(result_dir, '{}.json'.format(data_name))
    write_json(results, result_path)

Retrieved dataset for adult
Limited dataset to 1000 samples for testing
Applied category mapping
X_origin is replaced with a new version where categorical variables have been mapped to numerical values.
Data Size: (1000, 15)
Starting k-fold validation with k=5

Processing 5% missing data

Fold 1
Generated incomplete data with missing values for fold 1:
Original data with missing values (subset):
[[41 0 264663 2 10 nan nan 2 0 0 nan 3900 40 0 0]
 [38 0 150601 3 6 2 4 1 0 1 0 nan 40 0 0]
 [74 1 88638 4 16 3 2 3 0 0 nan 3683 20 0 1]
 [68 2 422013 0 9 1 nan 0 0 0 0 3683 40 0 0]
 [38 3 nan 5 15 3 2 0 0 nan 0 2824 45 0 1]]


100%|███████████████████████████████████████████| 15/15 [00:14<00:00,  1.03it/s]


Completed imputation for fold 1:
X_res with imputed values (subset):
[[41 0 264663 2 10 4 2 2 0 0 0.42824387550354004 3900 40 0 0]
 [38 0 150601 3 6 2 4 1 0 1 0 2021.291259765625 40 0 0]
 [74 1 88638 4 16 3 2 3 0 0 -1.2451661825180054 3683 20 0 1]
 [68 2 422013 0 9 1 3 0 0 0 0 3683 40 0 0]
 [38 3 136020.734375 5 15 3 2 0 0 0 0 2824 45 0 1]]
Metrics for this fold: {'nrmse': 0.3958, 'col_1_error_rate': 0.425, 'col_3_error_rate': 0.1628, 'col_5_error_rate': 0.2308, 'col_6_error_rate': 0.6667, 'col_7_error_rate': 0.1316, 'col_8_error_rate': 0.0513, 'col_9_error_rate': 0.0385, 'col_13_error_rate': 0.0435, 'col_14_error_rate': 0.0789, 'total_error_rate': 0.1818, 'clf_aucroc': 0.8556}

Fold 2
Generated incomplete data with missing values for fold 2:
Original data with missing values (subset):
[[82 0 132870 0 9 0 0 0 0 0 0 4356 18 0 0]
 [54 0 140359 1 4 1 1 1 0 0 0 3900 40 0 0]
 [41 0 264663 2 10 nan 2 2 0 0 0 3900 40 0 0]
 [34 0 216864 0 9 1 nan 1 0 0 0 3770 45 0 0]
 [38 0 150601 3 6 2 4 1 0 

100%|███████████████████████████████████████████| 15/15 [00:13<00:00,  1.09it/s]


Completed imputation for fold 2:
X_res with imputed values (subset):
[[82 0 132870 0 9 0 0 0 0 0 0 4356 18 0 0]
 [54 0 140359 1 4 1 1 1 0 0 0 3900 40 0 0]
 [41 0 264663 2 10 3 2 2 0 0 0 3900 40 0 0]
 [34 0 216864 0 9 1 9 1 0 0 0 3770 45 0 0]
 [38 0 150601 3 6 2 4 1 0 1 0 3770 40 0 0]]
Metrics for this fold: {'nrmse': 0.5191, 'col_1_error_rate': 0.3235, 'col_3_error_rate': 0.5, 'col_5_error_rate': 0.2791, 'col_6_error_rate': 0.5135, 'col_7_error_rate': 0.1667, 'col_8_error_rate': 0.0667, 'col_9_error_rate': 0.1579, 'col_13_error_rate': 0.0526, 'col_14_error_rate': 0.0811, 'total_error_rate': 0.233, 'clf_aucroc': 0.8996}

Fold 3
Generated incomplete data with missing values for fold 3:
Original data with missing values (subset):
[[82 0 132870 0 9 0 0 0 0 0 0 4356 18 0 0]
 [nan 0 140359 1 4 1 1 1 0 0 0 3900 40 0 0]
 [41 0 264663 2 10 2 2 2 0 0 0 3900 40 0 0]
 [34 0 216864 0 9 1 3 1 0 0 0 3770 45 0 0]
 [38 0 150601 3 6 nan 4 1 0 nan 0 3770 40 0 0]]


...

Completed imputation for fold 3:
X_res with imputed values (subset):
[[82 0 132870 0 9 0 0 0 0 0 0 4356 18 0 0]
 [54 0 190719.109375 1 8.321946144104004 1 1 1 0 0 0 2050.2861328125 40 0
  0]
 [41 0 264663 2 10 2 2 2 0 0 0 2096.576416015625 40 0 0]
 [34 0 216864 0 9 1 4 1 0 0 0 3770 45 0 0]
 [34.659175872802734 0 150601 3 8.091200828552246 2 4 1 0 1 0 3770 40 0 0]]
Metrics for this fold: {'nrmse': 0.4551, 'col_1_error_rate': 0.306, 'col_3_error_rate': 0.3064, 'col_5_error_rate': 0.2424, 'col_6_error_rate': 0.6347, 'col_7_error_rate': 0.1818, 'col_8_error_rate': 0.0765, 'col_9_error_rate': 0.1465, 'col_13_error_rate': 0.0432, 'col_14_error_rate': 0.1849, 'total_error_rate': 0.2359, 'clf_aucroc': 0.8643}

Fold 4
Generated incomplete data with missing values for fold 4:
Original data with missing values (subset):
[[82 nan 132870 0 nan 0 0 0 0 0 0 4356 nan 0 0]
 [nan 0 140359 1 4 nan 1 1 0 0 0 3900 40 0 0]
 [41 0 nan 2 10 2 2 2 0 0 0 3900 nan nan nan]
 [34 0 216864 0 9 1 3 1 nan 0 0 3770 45

100%|███████████████████████████████████████████| 15/15 [00:14<00:00,  1.02it/s]


Completed imputation for fold 4:
X_res with imputed values (subset):
[[82 0 132870 0 9.851913452148438 0 0 0 0 0 0 4356 32.657283782958984 0 0]
 [45.85426330566406 0 140359 1 4 4 1 1 0 0 0 3900 40 0 0]
 [41 0 154493.859375 2 10 2 2 2 0 0 0 3900 39.72895050048828 0 1]
 [34 0 216864 0 9 1 3 1 0 0 0 3770 45 0 0]
 [38 0 199159.953125 3 6 2 4 0 0 1 0 3770 40 0 0]]
Metrics for this fold: {'nrmse': 0.5788, 'col_1_error_rate': 0.3273, 'col_3_error_rate': 0.3493, 'col_5_error_rate': 0.2875, 'col_6_error_rate': 0.6415, 'col_7_error_rate': 0.1726, 'col_8_error_rate': 0.0643, 'col_9_error_rate': 0.1267, 'col_13_error_rate': 0.0562, 'col_14_error_rate': 0.1418, 'total_error_rate': 0.2378, 'clf_aucroc': 0.8484}

Fold 5
Generated incomplete data with missing values for fold 5:
Original data with missing values (subset):
[[82 0 132870 nan 9 0 0 nan 0 0 0 nan 18 nan nan]
 [54 0 nan 1 4 nan 1 1 0 nan 0 nan 40 0 0]
 [34 nan nan 0 9 1 nan 1 nan 0 0 3770 nan 0 nan]
 [74 1 88638 nan 16 3 2 3 nan 0 nan nan 2

100%|███████████████████████████████████████████| 15/15 [00:14<00:00,  1.03it/s]


Completed imputation for fold 5:
X_res with imputed values (subset):
[[82 0 132870 0 9 0 0 0 0 0 0 2054.958984375 18 0 1]
 [54 0 247192.234375 1 4 4 1 1 0 1 0 2155.4970703125 40 0 0]
 [34 0 241184.921875 0 9 1 4 1 0 0 0 3770 47.14485168457031 0 0]
 [74 1 88638 4 16 3 2 3 0 0 -0.34388917684555054 2008.1453857421875 20 0
  1]
 [68 2 422013 0 9 1 4 0 0 0 0 1996.4923095703125 40 0 0]]
Metrics for this fold: {'nrmse': 0.5286, 'col_1_error_rate': 0.2733, 'col_3_error_rate': 0.3879, 'col_5_error_rate': 0.2917, 'col_6_error_rate': 0.6923, 'col_7_error_rate': 0.1657, 'col_8_error_rate': 0.046, 'col_9_error_rate': 0.1677, 'col_13_error_rate': 0.06, 'col_14_error_rate': 0.1718, 'total_error_rate': 0.2438, 'clf_aucroc': 0.878}
Average nrmse for 20% missing data: 0.52108
Average col_1_error_rate for 20% missing data: 0.32345999999999997
Average col_3_error_rate for 20% missing data: 0.31062
Average col_5_error_rate for 20% missing data: 0.25122
Average col_6_error_rate for 20% missing data: 0.67754

In [ ]:
[38 nan 150601 3 6 nan 4 1 0 1 0 3770 nan 0 0]]
[74 1 88638 4 16 3 2 3 0 0 0 3683 20 0 1]
[41 0 264663 2 10 3 2 2 0 0 0 3900 40 0 0]
[41 0 264663 2 10 nan nan 2 0 0 nan 3900 40 0 0]
[41 0 264663 2 10 4 2 2 0 0 0.42824387550354004 3900 40 0 0